# Exploration of Cornell-MOE 

In [1]:
import sys

In [2]:
#Custom package environment
sys.path.insert(0,'/KIMEL/tigrlab/projects/jjeyachandra/gmsh-sdk/lib/')
sys.path.insert(0,'/home/jjeyachandra/simnibs_2.1.2/miniconda2/envs/simnibs_env/lib/python2.7/site-packages')
sys.path.insert(0,'/home/jjeyachandra/simnibs_2.1.2/Python_modules/src')

In [3]:
import numpy as np
import os

In [4]:
#Add cornell library
sys.path.insert(0, '/projects/jjeyachandra/Cornell-MOE/')

In [5]:
from examples import synthetic_functions

#Define an objective function using their example func
objective_func = synthetic_functions.Rosenbrock()

In [6]:
from moe.optimal_learning.python.cpp_wrappers.domain import TensorProductDomain as cTensorProductDomain
from moe.optimal_learning.python.python_version.domain import TensorProductDomain
from moe.optimal_learning.python.geometry_utils import ClosedInterval

#Make an n-dimensional domain containing closed intervals
search_domain = TensorProductDomain([ClosedInterval(bound[0],bound[1]) for bound in objective_func._search_domain])
cpp_search_domain = cTensorProductDomain([ClosedInterval(bound[0],bound[1]) for bound in objective_func._search_domain])

In [7]:
#Array to fill in with initial points
init_pts = np.zeros( (objective_func._num_init_pts, objective_func._dim) )
init_pts[:, :objective_func._dim] = search_domain.generate_uniform_random_points_in_domain(
                                            objective_func._num_init_pts)
print(init_pts)

[[ 1.4702256  -0.3150478 ]
 [-0.07813419 -0.76792158]
 [-1.55459385  1.64201219]]


In [8]:
#Evaluate at init pts using noise-less sampling
observations = np.array([objective_func.evaluate_true(p)[0] for p in init_pts])
print(observations)

[ 613.58137425   61.07408108   66.54968349]


In [9]:
from moe.optimal_learning.python.data_containers import HistoricalData, SamplePoint

#Store observations into a data_container and display
init_data = HistoricalData(dim = objective_func._dim, num_derivatives=0)
init_data.append_sample_points([SamplePoint(pt, o, 0.0) for pt,o in zip(init_pts,observations)])

In [10]:
print(init_data)

[_BaseSamplePoint(point=array([ 1.4702256, -0.3150478]), value=array([ 613.58137425]), noise_variance=0.0),
 _BaseSamplePoint(point=array([-0.07813419, -0.76792158]), value=array([ 61.07408108]), noise_variance=0.0),
 _BaseSamplePoint(point=array([-1.55459385,  1.64201219]), value=array([ 66.54968349]), noise_variance=0.0)]


Setting up the log-likelihood minimization of $\mathcal{GP}$ model

In [11]:
from moe.optimal_learning.python.cpp_wrappers.log_likelihood_mcmc import GaussianProcessLogLikelihoodMCMC
from moe.optimal_learning.python.default_priors import DefaultPrior
#Priors for (?)
# l : D length scales
# mu : len(y) number of means
# v : number of noise values
# t : covariance amplitude
# Dimensions = D + (len(y)) + t
# Noise Terms = len(y)
prior = DefaultPrior(n_dims=objective_func._dim + 2,num_noise=1)
gp_ll = GaussianProcessLogLikelihoodMCMC(historical_data=init_data,derivatives = [],
                                        prior=prior, chain_length=1000, burnin_steps=2000,
                                        n_hypers=2**4, noisy=False, rng=20)
## n_hypers is a lower-limit on the number of MCMC chains to use
gp_ll.train()

In [12]:
from moe.optimal_learning.python.python_version.optimization import GradientDescentOptimizer, GradientDescentParameters
from moe.optimal_learning.python.cpp_wrappers.optimization import GradientDescentOptimizer as cGDOpt
from moe.optimal_learning.python.cpp_wrappers.optimization import GradientDescentParameters as cGDParams
sgd_params = cGDParams(num_multistarts=200, 
                       max_num_steps=50,
                       max_num_restarts=2,
                       num_steps_averaged=4,
                       gamma=0.7,
                       pre_mult=1.0,
                       max_relative_change=0.5,
                       tolerance=1.0e-10)

In [13]:
from moe.optimal_learning.python.cpp_wrappers.expected_improvement import ExpectedImprovement
from moe.optimal_learning.python.cpp_wrappers.expected_improvement import multistart_expected_improvement_optimization as meio

def gen_sample_from_qei(gp,search_domain,sgd_params,num_samples, num_mc=1e4, lhc_iter=2e4):
    
    qEI = ExpectedImprovement(gaussian_process=gp, num_mc_iterations=int(num_mc))
    optimizer = cGDOpt(search_domain, qEI, sgd_params, int(lhc_iter))
    points_to_sample = []
    ei_list = []
    
    points_to_sample.append(meio(optimizer, None, num_samples,use_gpu=False,which_gpu=0,
                                max_num_threads=8))
    
    
    qEI.set_current_point(points_to_sample[0])
    ei_list.append(qEI.compute_expected_improvement())
    return points_to_sample[0], ei_list[0]

Loop until convergence criterion/satisfactory value reached (test to make sure it works)

Implementation of process:

In [14]:
from IPython.display import clear_output, display
num_iters = 50
optimum = objective_func._min_value

In [15]:
objective_func.evaluate_true(np.array([1,1]))

array([ 0.,  0.,  0.])

In [17]:
#Generate optimal points to sample
for i in np.arange(0,20):   
    
    points_to_sample,ei = gen_sample_from_qei(gp_ll.models[0], cpp_search_domain, sgd_params=sgd_params,
                                              num_samples=8,num_mc=2**10)
    #Evaluate underlying objective function at these points
    sampled_points = [objective_func.evaluate_true(x)[0] for x in points_to_sample]

    #Build list of sampled points
    evidence_tup = [ SamplePoint(c,v,0.0) for c,v in zip(points_to_sample, sampled_points)] 

    #Add to GPLogLikelihoodMaximization and re-train new model
    gp_ll.add_sampled_points(evidence_tup)
    gp_ll.train()

    #Pull model being trained and compute errors
    gp = gp_ll.models[1]
    min_point = np.argmin(gp._points_sampled_value)
    min_val = np.min(gp._points_sampled_value)
    best_coord = gp.get_historical_data_copy().points_sampled[min_point,:]
    error = np.abs(min_val - optimum)

    #Print out useful information
    clear_output()
    print('Recommended Points:')
    print(points_to_sample)
    print('Expected Improvement: {}'.format(ei))
    print('Current Best:')
    print('Coord:', best_coord)
    print('f(x*)=', min_val)
    print('Error:', error)

SingularMatrixException: SingularMatrixException: 27 x 27 matrix is singular; 17-th leading minor is not SPD.
Covariance matrix (K) singular. Check for duplicate points_sampled (with 0 noise) and/or extreme hyperparameter values. void optimal_learning::GaussianProcess::RecomputeDerivedVariables(bool) (/mnt/tigrlab/projects/jjeyachandra/Cornell-MOE/moe/optimal_learning/cpp/gpp_math.cpp: 495)

Stuck in local optima for some reason??????